# Description and Goal

This project performs cleaning, EDA, and modelling to the BlueBikes bicycle sharing system data from January 2015 to June 2025 found [here](https://bluebikes.com/system-data)



| Column (From website)      |
| ----------- |
| Trip Duration (seconds) |
| Start Time and Date |
| Stop Time and Date |
| Start Station Name & ID |
| End Station Name & ID |
| Bike ID |
| User Type (Casual = Single Trip or Day Pass user; Member = Annual or Monthly Member) |









# Load and Clean Data

Step 1: Preprocessing (in Python/Pandas)

Let's load data. Lots of files from the website that we need to standardize column names for and concatenate into one csv file?
- Loop through CSVs, inspect column names, standardize them.
- Concatenate all into one big DataFrame.
- Clean data (e.g., fix datetime parsing, column types, missing values).

In [ ]:
import pandas as pd

In [ ]:
start_path = "/Users/ellawang/Documents/GitHub/bike_csv_files/"
old_end_path = "-hubway-tripdata.csv"
new_end_path = "-bluebikes-tripdata.csv"
yr_15 = ["2015" + str(i).zfill(2) + old_end_path for i in range(1, 13)]
yr_16 = ["2016" + str(i).zfill(2) + old_end_path for i in range(1, 13)]
yr_17 = ["2017" + str(i).zfill(2) + old_end_path for i in range(1, 13)]
yr_18_1 = ["2018" + str(i).zfill(2) + old_end_path for i in range(1, 5)] 
    # note 1801-1803 i had to manually replace _ with - in the names
    # after 1805 hubway-->bluebikes
yr_18_2 = ["2018" + str(i).zfill(2) + new_end_path for i in range(5, 13)] 
yr_19 = ["2019" + str(i).zfill(2) + new_end_path for i in range(1, 13)]
yr_20 = ["2020" + str(i).zfill(2) + new_end_path for i in range(1, 13)]
yr_21 = ["2021" + str(i).zfill(2) + new_end_path for i in range(1, 13)]
yr_22 = ["2022" + str(i).zfill(2) + new_end_path for i in range(1, 13)]
yr_23 = ["2023" + str(i).zfill(2) + new_end_path for i in range(1, 13)]
yr_24 = ["2024" + str(i).zfill(2) + new_end_path for i in range(1, 13)]
yr_25 = ["2025" + str(i).zfill(2) + new_end_path for i in range(1, 7)]
pathways = yr_15 + yr_16 + yr_17 + yr_18_1 + yr_18_2 + yr_19 + yr_20 + yr_21 + yr_22 + yr_23 + yr_24 + yr_25

# condense this shii

In [ ]:
# # give us a peak into the columns and formats/datatypes of each file
# num_total_rows = 0
# col_count = {}
# for path in pathways:
#     df = pd.read_csv(start_path + path)
#     print(f'{path}: {df.columns} : {df.shape[0]} rows')
#     num_total_rows += df.shape[0]
#     print(df.iloc[0])
#     for col in df.columns:
#         if col not in col_count:
#             col_count[col] = 0
#         col_count[col] += 1

# print(col_count)
# print(f'Num total rows: {num_total_rows}')

# # saved to "output.txt" so don't have to re-run

We see inconsistent naming conventions. investigated into output of the print statements printing one line from each file to see which columns are the same and of those which are reformatted and also which columns like dropped before or after a certain point

Column names in 99 files (201501 until 202303 (including final yr/mo))
- 'tripduration': 99 (ends in 202303) (e.g. 1105) -- DROPPING tentatively?
- 'bikeid': 99, (ends in 202304) (e.g. 6680) -- DROPPING tentatively?
- 'starttime': 99 (turns into started_at beg. 202304) (e.g. 2023-03-01 00:00:44.1520 --> 2023-04-13 13:49:59)
- 'stoptime': 99 (turns into ended_at beg. 202304)
- 'start station id': 99 (turns into start_station_id beg. 202304) (e.g. 386 --> A32011)
- 'start station name': 99, (turns into start_station_name beg. 202304) (e.g. Central Square at Mass Ave / Essex St --> seems to stay same!)
- start station latitude': 99, (turns into start_lat beg. 202304) (e.g. 42.368605 --> 42.363713 stays the same!)
- 'start station longitude': 99, (turns into start_lng beg. 202304) (same)
- 'end station id': 99, (turns into end_station_id beg. 202304) (e.g. 386 --> A32011 aka same)
- 'end station name': 99, (turns into end_station_name beg. 202304) (same)
- 'end station latitude': 99, (turns into end_lat beg. 202304) (same)
- 'end station longitude': 99, (turns into end_lng beg. 202304) (same)
- 'usertype': 99(turns into member_casual beg. 202304) (e.g. Customer or Subscriber --> member or casual)

Column names in 64 files (201501 until 202004)
- 'birth year': 64 (e.g. 1984) -- DROPPING
- 'gender': 64 (e.g. 0 or 1 or 2) -- DROPPING

Column names in 35 files (202005 until 202303)
- 'postal code': 35 (e.g. 02118 or NaN) -- DROPPING

Column names in 27 files (202304 to 202506)
- 'ride_id': 27 (begins 202304) (e.g. 0093AA5E7E3E0158) -- DROPPING
- 'rideable_type': 27, (begins 202304) (e.g. docked_bike or classic_bike or electric_bike) -- DROPPING tentatively?

dropping columns: I will delete **birth year, gender, postal code** since those are present in only half or fewer of the rows and not the most imformative. I will drop **ride_id** since not informative and just distinguishes rides from each other, **bikeid** because I don't care too much about particular bike (not sure about htis assumption hm), dropping **tripduration** bc that can be deduced from starttime and endtime (i'll engineer a new col after this). 

also will drop **start station id** and **end station id** bc the format changes halfway and redundant with start and end station name

for now, i will drop **rideable type** bc it's only in 27 rows... however this is a meaningful var to predict other things so will do more research (maybe bluebikes only started offering e bikes a certain year and prior to that there was only classic bike... also idk the diff between classic and docked bike lol so will look into that later but for now drop?)

(might need to rewrite/move) then i'll rename columns, standardize formatting, and visualize with EDA as well as missing values before i decide how to go about filling in missing values

In [ ]:
# this takes 1 min 15 sec to run ish

# drop those columns - 
def load_and_clean_csv(filepath):
    
    # read_csv
    df = pd.read_csv(filepath)
    
    # rename cols as needed pass in dict
    renames = {
        'starttime': 'started_at',
        'stoptime': 'ended_at',
        'start station id': 'start_station_id',
        'start station name': 'start_station_name',
        'start station latitude': 'start_lat',
        'start station longitude': 'start_lng',
        'end station id': 'end_station_id',
        'end station name': 'end_station_name',
        'end station latitude': 'end_lat',
        'end station longitude': 'end_lng',
        'usertype' : 'member_casual'
    }
    
    df.rename(columns = renames, inplace=True)
    
    
    # get a subset of columns wanted
    keep_columns = ['started_at', 'ended_at', 'start_station_name',
       'start_station_id', 'start_lat', 'start_lng', 'end_station_id',
       'end_station_name', 'end_lat', 'end_lng', 'member_casual']
    
    df = df[keep_columns]
    
    return df
    
    # for path in pathways:
    #     df = pd.read_csv(start_path + path)
    #     print(f'{path}: {df.columns} : {df.shape[0]} rows')
    #     num_total_rows += df.shape[0]
    #     print(df.iloc[0])
    #     for col in df.columns:
    #         if col not in col_count:
    #             col_count[col] = 0
    #         col_count[col] += 1

    # print(col_count)
    # print(f'Num total rows: {num_total_rows}')
    # return df

# get list of all pathways (this is pathways from prev code cell)

# # get list of dataframes
dfs_list = [load_and_clean_csv(start_path + pathway) for pathway in pathways]

# # concat and get a list of that funciton applied to each pathways
original_df = pd.concat(dfs_list, ignore_index=True)

In [ ]:
big_df = original_df.copy()
big_df

### gotta fix the datetime for started_at and ended_at, which are diff formats before and after certain index

**2023-03-01 00:00:44.1520** --> **2023-04-13 13:49:59** beginning 202304

- 'start station id': 99 (turns into start_station_id beg. 202304) (e.g. 386 --> A32011)
- 'start station name': 99, (turns into start_station_name beg. 202304) (e.g. Central Square at Mass Ave / Essex St --> seems to stay same!)
- start station latitude': 99, (turns into start_lat beg. 202304) (e.g. 42.368605 --> 42.363713 stays the same!)
- 'start station longitude': 99, (turns into start_lng beg. 202304) (same)
- 'end station id': 99, (turns into end_station_id beg. 202304) (e.g. 386 --> A32011 aka same)
- 'end station name': 99, (turns into end_station_name beg. 202304) (same)
- 'end station latitude': 99, (turns into end_lat beg. 202304) (same)
- 'end station longitude': 99, (turns into end_lng beg. 202304) (same)
- 'usertype': 99(turns into member_casual beg. 202304) (e.g. Customer or Subscriber --> member or casual)

In [ ]:
big_df.info()

need to convert end_lat and end_long to floats

In [ ]:
# ValueError: could not convert string to float: '\\N'
import numpy as np
big_df.replace('\\N', np.nan, inplace=True)

In [ ]:
big_df['end_lat'] = big_df['end_lat'].astype(float)
big_df['end_lng'] = big_df['end_lng'].astype(float)

In [202]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27065610 entries, 0 to 27065609
Data columns (total 11 columns):
 #   Column              Dtype         
---  ------              -----         
 0   started_at          datetime64[ns]
 1   ended_at            datetime64[ns]
 2   start_station_name  object        
 3   start_station_id    object        
 4   start_lat           float64       
 5   start_lng           float64       
 6   end_station_id      object        
 7   end_station_name    object        
 8   end_lat             float64       
 9   end_lng             float64       
 10  member_casual       object        
dtypes: datetime64[ns](2), float64(4), object(5)
memory usage: 2.2+ GB


good!

In [ ]:
big_df.describe()

In [ ]:
big_df.isna().sum()

In [ ]:
# takes like 40 seconds to run

# view rows with at least one NaN value
big_df[big_df.isna().any(axis=1)]

# notice a lot of these bike rides lasted over a day ... maybe they were abandoned/lost/stolen/data entry error/broken dock so will drop probably
# to test let's engineer a new trip_duration feature from started_at and ended_at

In [ ]:
# big_df[big_df.isna().any(axis=1)].index

In [ ]:
# takes like 22 seconds to run

null_indices = big_df[big_df.isna().any(axis=1)].index.tolist()

In [ ]:
null_indices[0:5]

In [ ]:
test_rows = big_df.iloc[:2]
test_rows

In [ ]:
type(test_rows.iloc[0]['started_at'])

In [ ]:
# cast string to datetime
pd.to_datetime(test_rows.iloc[0]['started_at'])


In [ ]:
# 2023-03-01 00:00:44.1520** --> **2023-04-13 13:49:59** beginning 202304

# casting started_at and ended_at as datetime

# apply to columns
big_df['started_at'] = pd.to_datetime(big_df['started_at'])
big_df['ended_at'] = pd.to_datetime(big_df['ended_at'])

In [ ]:
# engineer new duration column
big_df['duration'] = big_df['ended_at'] - big_df['started_at']
big_df.head()

In [ ]:
big_df.info()

good!

In [ ]:
big_df.iloc[0:2]

In [ ]:
big_df.iloc[-2:]

now need to deal with station name and ids columns...

In [190]:
big_df['start_station_name'].value_counts()

MIT at Mass Ave / Amherst St                       609332
Central Square at Mass Ave / Essex St              470967
Harvard Square at Mass Ave/ Dunster                389056
MIT Pacific St at Purrington St                    311547
MIT Stata Center at Vassar St / Main St            310863
                                                    ...  
MTL-ECO4-01                                             1
Meaney Playground - Pleasant st at E Cottage St         1
Swan Pl. at Minuteman Bikeway                           1
Medford St. at Warren St.                               1
Jamaicaway at Perkins St                                1
Name: start_station_name, Length: 831, dtype: int64

In [191]:
big_df['end_station_name'].value_counts()

MIT at Mass Ave / Amherst St               597449
Central Square at Mass Ave / Essex St      471718
Harvard Square at Mass Ave/ Dunster        401820
Ames St at Main St                         339129
MIT Stata Center at Vassar St / Main St    326066
                                            ...  
Swan Pl. at Minuteman Bikeway                   1
Warehouse Lab PBSC                              1
Lab PBSC                                        1
Mobile Temporary Station 1                      1
Jamaicaway at Perkins St                        1
Name: end_station_name, Length: 832, dtype: int64

In [192]:
big_df['start_station_id'].value_counts()

67        419951
68        324437
74        257707
80        235133
22        228985
           ...  
158            3
230            3
E32002         1
382            1
383            1
Name: start_station_id, Length: 1103, dtype: int64

In [193]:
big_df['end_station_id'].value_counts()

67        409623
68        324755
74        266312
80        252025
107       229632
           ...  
229            5
153            3
158            3
382            2
E32002         1
Name: end_station_id, Length: 1135, dtype: int64

ick. there's an uneven number of start vs. end station names and ids... and the number of names and ids do not match... let's look into other files included which give like metadata i think! or like data about stations and our map essentially

In [ ]:
stations_df = pd.read_csv('/Users/ellawang/Documents/GitHub/bike_etc_files/current_bluebikes_stations.csv')
stations_df

In [256]:
relative_path = '/Users/ellawang/Documents/GitHub/bike_etc_files/'
trip_start = 'hubway_Trips_20'

hubway_stations_2011_2016 = relative_path + 'Hubway_Stations_2011_2016.csv'
hubway_stations_2017 = relative_path + 'Hubway_Stations_as_of_July_2017.csv'
trips_2011 = relative_path + trip_start + '11.csv'
trips_2012 = relative_path + trip_start + '12.csv'
trips_2013 = relative_path + trip_start + '13.csv'
trips_2014_1 = relative_path + trip_start + '14_1.csv'
trips_2014_2 = relative_path + trip_start + '14_2.csv'
previous_hubway_stations_2017 = relative_path + 'previous_Hubway_Stations_as_of_July_2017.csv'

pre_2015_paths = [trips_2011, trips_2012, trips_2013, trips_2014_1, trips_2014_2]

In [187]:
hubway_stations_2011_2016_df = pd.read_csv(hubway_stations_2011_2016)
hubway_stations_2011_2016_df # 187

,Station,Station ID,Latitude,Longitude,Municipality,# of Docks
0,Fan Pier,A32000,42.353287,-71.044389,Boston,15
1,Union Square - Brighton Ave. at Cambridge St.,A32001,42.353334,-71.137313,Boston,15
2,Agganis Arena - 925 Comm Ave.,A32002,42.351246,-71.115639,Boston,19
3,B.U. Central - 725 Comm. Ave.,A32003,42.350406,-71.108279,Boston,11
4,Longwood Ave / Binney St,A32004,42.338629,-71.106500,Boston,15
...,...,...,...,...,...,...
182,Packard Ave / Powderhouse Blvd,S32009,42.404490,-71.123413,Somerville,15
183,Somerville Hospital at Highland Ave / Crocker St,S32010,42.390820,-71.109420,Somerville,15
184,Teele Square at 239 Holland St,S32011,42.402763,-71.126908,Somerville,15
185,Magoun Square at Trum Field,S32013,42.398365,-71.107818,Somerville,15


In [189]:
hubway_stations_2017_df = pd.read_csv(hubway_stations_2017)
hubway_stations_2017_df.info() # 330

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Number       330 non-null    object 
 1   Name         330 non-null    object 
 2   Latitude     330 non-null    float64
 3   Longitude    330 non-null    float64
 4   District     330 non-null    object 
 5   Public       330 non-null    object 
 6   Total docks  330 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 18.2+ KB


In [196]:
trips_2011_df = pd.read_csv(trips_2011)
print(trips_2011_df.columns) #140,974
trips_2011_df.iloc[0]

Index(['Duration', 'Start date', 'End date', 'Start station number',
       'Start station name', 'End station number', 'End station name',
       'Bike number', 'Member type', 'Zip code', 'Gender'],
      dtype='object')


/var/folders/2d/lf9lppb564bfjnr36gbpwvz00000gn/T/ipykernel_98569/562385137.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  hubway_trips_2011_df = pd.read_csv(hubway_trips_2011)


Duration                                                 1712320
Start date                                      11/30/2011 23:58
End date                                          12/1/2011 0:26
Start station number                                      D32005
Start station name      Boston Public Library - 700 Boylston St.
End station number                                        D32011
End station name                       Stuart St. at Charles St.
Bike number                                               B00056
Member type                                               Member
Zip code                                                    2116
Gender                                                      Male
Name: 0, dtype: object

In [197]:
trips_2012_df = pd.read_csv(trips_2012)
print(trips_2012_df.columns) # 530,374
trips_2012_df.iloc[0]

Index(['Duration', 'Start date', 'End date', 'Start station number',
       'Start station name', 'End station number', 'End station name',
       'Bike number', 'Member type', 'Zip code', 'Gender'],
      dtype='object')


Duration                                  1633357
Start date                       11/28/2012 23:58
End date                          11/29/2012 0:25
Start station number                       B32005
Start station name        Christian Science Plaza
End station number                         D32011
End station name        Stuart St. at Charles St.
Bike number                                T01350
Member type                                Member
Zip code                                     2116
Gender                                       Male
Name: 0, dtype: object

In [198]:
trips_2013_df = pd.read_csv(trips_2013)
print(trips_2013_df.columns) # 906,876
trips_2013_df.iloc[0]

Index(['Duration', 'Start date', 'End date', 'Start station number',
       'Start station name', 'End station number', 'End station name',
       'Bike number', 'Member type', 'Zip code', 'Gender'],
      dtype='object')


/var/folders/2d/lf9lppb564bfjnr36gbpwvz00000gn/T/ipykernel_98569/1171236407.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  hubway_trips_2013_df = pd.read_csv(hubway_trips_2013)


Duration                                                          249396
Start date                                              12/31/2013 23:25
End date                                                12/31/2013 23:29
Start station number                                              M32037
Start station name                                    Ames St at Main St
End station number                                                M32002
End station name        One Kendall Square at Hampshire St / Portland St
Bike number                                                       T01163
Member type                                                       Member
Zip code                                                            2141
Gender                                                            Female
Name: 0, dtype: object

In [199]:
trips_2014_1_df = pd.read_csv(trips_2014_1)
print(trips_2014_1_df.columns) # 417,473
trips_2014_1_df.iloc[0]

Index(['Duration', 'Start date', 'End date', 'Start station number',
       'Start station name', 'End station number', 'End station name',
       'Bike number', 'Member type', 'Zip code', 'Gender'],
      dtype='object')


/var/folders/2d/lf9lppb564bfjnr36gbpwvz00000gn/T/ipykernel_98569/2425993394.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  hubway_trips_2014_1_df = pd.read_csv(hubway_trips_2014_1)


Duration                                                       483653
Start date                                            6/30/2014 20:06
End date                                              6/30/2014 20:14
Start station number                                           M32026
Start station name          359 Broadway - Broadway at Fayette Street
End station number                                             M32010
End station name        Inman Square at Vellucci Plaza / Hampshire St
Bike number                                                    B00082
Member type                                                    Member
Zip code                                                         2139
Gender                                                           Male
Name: 0, dtype: object

In [200]:
trips_2014_2_df = pd.read_csv(trips_2014_2)
print(trips_2014_2_df.columns) # 767,241
trips_2014_2_df.iloc[0]

Index(['Duration', 'Start date', 'End date', 'Start station number',
       'Start station name', 'End station number', 'End station name',
       'Bike number', 'Member type', 'Zip code', 'Gender'],
      dtype='object')


/var/folders/2d/lf9lppb564bfjnr36gbpwvz00000gn/T/ipykernel_98569/3455016750.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  hubway_trips_2014_2_df = pd.read_csv(hubway_trips_2014_2)


Duration                                                397101
Start date                                    12/31/2014 23:58
End date                                         1/1/2015 0:05
Start station number                                    B32004
Start station name        Aquarium Station - 200 Atlantic Ave.
End station number                                      D32022
End station name        TD Garden - Causeway at Portal Park #1
Bike number                                             T01335
Member type                                             Member
Zip code                                                2148.0
Gender                                                    Male
Name: 0, dtype: object

duration is in milliseconds!

In [201]:
previous_hubway_stations_2017_df = pd.read_csv(previous_hubway_stations_2017)
print(previous_hubway_stations_2017_df.columns) # 281
previous_hubway_stations_2017_df.iloc[0]

Index(['Station ID', 'Station', 'Latitude', 'Longitude', 'Municipality',
       'publiclyExposed', '# of Docks'],
      dtype='object')


Station ID                   A32019
Station            175 N Harvard St
Latitude                  42.363796
Longitude                -71.129164
Municipality                 Boston
publiclyExposed                   1
# of Docks                       18
Name: 0, dtype: object

okkkk so this is super msessy and incocnsitnet. the pre-2015 ones do not have lat and long so will concatenate those later. first i'm going to get a clean, reliable master station reference from all 2015+ rows

In [204]:
big_df.iloc[0]

started_at                                          2015-01-01 00:21:44
ended_at                                            2015-01-01 00:30:47
start_station_name                                Porter Square Station
start_station_id                                                    115
start_lat                                                     42.387995
start_lng                                                    -71.119084
end_station_id                                                       96
end_station_name      Cambridge Main Library at Broadway / Trowbridg...
end_lat                                                       42.373379
end_lng                                                      -71.111075
member_casual                                                Subscriber
Name: 0, dtype: object

In [208]:
start_stations = big_df[['start_station_name', 'start_station_id', 'start_lat', 'start_lng']].rename(
    columns = {
        'start_station_name' : 'station_name',
        'start_station_id' : 'station_id',
        'start_lat' : 'lat',
        'start_lng' : 'long'
    }
)

In [209]:
end_stations = big_df[['end_station_name', 'end_station_id', 'end_lat', 'end_lng']].rename(
    columns = {
        'end_station_name' : 'station_name',
        'end_station_id' : 'station_id',
        'end_lat' : 'lat',
        'end_lng' : 'long'
    }
)

In [210]:
station_master = pd.concat([start_stations, end_stations]) # concat
print('pre dropna len:', station_master.shape[0])
station_master.dropna(inplace=True) # drop any nas
print('post dropna len:', station_master.shape[0])
station_master.drop_duplicates(inplace=True) # drop duplicates
print('post drop duplicate len:', station_master.shape[0])

pre dropna len: 54131220
post dropna len: 54100150
post drop duplicate len: 906926


In [231]:
station_master.sort_values(by='station_name').head(150)['station_id'].unique()

array(['H32007'], dtype=object)

In [ ]:
station_master = station_master.groupby('station_name').agg({
    'station_id': lambda x: list(set(x)),
    'lat': 'mean',
    'long': 'mean'
})

In [249]:
station_master

,station_id,lat,long
station_name,,,
Broadway and Cabot,[H32007],42.402255,-71.019582
101 Smith Place,[M32093],42.392528,-71.150655
1200 Beacon St,"[452, K32015]",42.344050,-71.115093
160 Arsenal,"[W32006, 502]",42.364664,-71.175694
160 Arsenal St,[W32006],42.364692,-71.175489
...,...,...,...
Williams St at Washington St,"[D32040, 279]",42.306618,-71.107707
Wilson Square,"[S32005, 99]",42.385608,-71.113962
Winthrop Circle,[F32014],42.421342,-71.117536


In [252]:
station_master.reset_index(inplace = True)

great! next i'm going to explore any inconsistencies or outliers etc. and resolve as needed... ambiguous task. and map the stations to make sure it seems reasonable

following [this link](https://edg.it.com/python-interactive-mapping/how-to-create-interactive-maps-with-python-using-openstreetmap-and-plotly/) to make a map

In [253]:
station_master.describe()

,index,lat,long
count,834.000000,834.000000,834.000000
mean,416.500000,41.979471,-70.449686
std,240.899357,3.935258,6.601854
min,0.000000,0.000000,-73.566921
25%,208.250000,42.336452,-71.114289
50%,416.500000,42.353383,-71.083238
75%,624.750000,42.375260,-71.060859
max,833.000000,45.505086,0.000000


In [254]:
import plotly.express as px

fig = px.scatter_mapbox(
    station_master,
    lat = 'lat',
    lon = 'long',
    hover_name = 'station_name'
)
fig.update_layout(mapbox_style="open-street-map")
fig.show()

yay! it seems to match the blue bikes system map when i log into my app (obviously some of these stations discontinued but ok)

then i am first going to get all the pre 2015 ones into a dataframe and merge with the post 2014 one

| pre 2015 col name | e.g. format |
|-----------------------|-------------------------------------------------|
| Duration              | 397101                                          |
| Start date            | 12/31/2014 23:58                                |
| End date              | 1/1/2015 0:05                                   |
| Start station number  | B32004                                          |
| Start station name    | Aquarium Station - 200 Atlantic Ave.           |
| End station number    | D32022                                          |
| End station name      | TD Garden - Causeway at Portal Park #1         |
| Bike number           | T01335                                          |
| Member type           | Member                                          |
| Zip code              | 2148.0                                          |
| Gender                | Male                                            |



note: so first need to turn **Customer -> casual** and **Subscriber -> member**

In [262]:
def load_and_clean_csv2(filepath):
    
    # read_csv
    df = pd.read_csv(filepath)
    
    renames = {
        
        'Duration' : 'duration',
        'Start date': 'started_at',
        'End date': 'ended_at',
        'Start station number': 'start_station_id',
        'Start station name': 'start_station_name',
        # 'start station latitude': 'start_lat',
        # 'start station longitude': 'start_lng',
        'End station number': 'end_station_id',
        'End station name': 'end_station_name',
        # 'end station latitude': 'end_lat',
        # 'end station longitude': 'end_lng',
        'Member type' : 'member_casual'
    }
    
    df.rename(columns = renames, inplace=True)
    
        # columns we want!
    keep_columns = ['duration', 'started_at', 'ended_at',
                    'start_station_id', 'start_station_name',
                    'end_station_id', 'end_station_name', 'member_casual']
    
    # will drop Bike number, Zip code, and Gender
    # need to convert started_at and ended_at to datetime;
    # convert Member type (Customer -> casual; Subscriber -> member)
    
    # other note to drop all 4 long/lat columns from big_df to save space / remove redundy info
    
    df = df[keep_columns]
    return df

In [290]:
# get list of dataframes
dfs_list2 = [load_and_clean_csv2(pathway) for pathway in pre_2015_paths]

# TODO resolve warnings?

/var/folders/2d/lf9lppb564bfjnr36gbpwvz00000gn/T/ipykernel_98569/1107282679.py:4: DtypeWarning:

Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/2d/lf9lppb564bfjnr36gbpwvz00000gn/T/ipykernel_98569/1107282679.py:4: DtypeWarning:

Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/2d/lf9lppb564bfjnr36gbpwvz00000gn/T/ipykernel_98569/1107282679.py:4: DtypeWarning:

Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/2d/lf9lppb564bfjnr36gbpwvz00000gn/T/ipykernel_98569/1107282679.py:4: DtypeWarning:

Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.



In [291]:
# concat and get a list of that funciton applied to each pathways
pre_2015_df = pd.concat(dfs_list2, ignore_index=True)

In [292]:
pre_2015_df.head(1)

,duration,started_at,ended_at,start_station_id,start_station_name,end_station_id,end_station_name,member_casual
0,1712320,11/30/2011 23:58,12/1/2011 0:26,D32005,Boston Public Library - 700 Boylston St.,D32011,Stuart St. at Charles St.,Member


In [293]:
pre_2015_df.tail(1)

,duration,started_at,ended_at,start_station_id,start_station_name,end_station_id,end_station_name,member_casual
2762937,156284,7/1/2014 0:00,7/1/2014 0:03,M32002,One Kendall Square at Hampshire St / Portland St,M32037,Ames St at Main St,Member


1. need to convert started_at and ended_at to datetime;
2. convert duration from milliseconds to datetime
3. convert Member type (Customer -> casual; Subscriber -> member)

TODO other note to drop all 4 long/lat columns from big_df to save space / remove redundy info

In [294]:
# 1 - takes over 2 mins to run hm
pre_2015_df['started_at'] = pd.to_datetime(pre_2015_df['started_at'])
pre_2015_df['ended_at'] = pd.to_datetime(pre_2015_df['ended_at'])
pre_2015_df.iloc[0]

duration                                               1712320
started_at                                 2011-11-30 23:58:00
ended_at                                   2011-12-01 00:26:00
start_station_id                                        D32005
start_station_name    Boston Public Library - 700 Boylston St.
end_station_id                                          D32011
end_station_name                     Stuart St. at Charles St.
member_casual                                           Member
Name: 0, dtype: object

In [295]:
# 2
pre_2015_df['duration'] = pd.to_timedelta(pre_2015_df['duration'], unit = 'milliseconds')
pre_2015_df.head()

,duration,started_at,ended_at,start_station_id,start_station_name,end_station_id,end_station_name,member_casual
0,0 days 00:28:32.320000,2011-11-30 23:58:00,2011-12-01 00:26:00,D32005,Boston Public Library - 700 Boylston St.,D32011,Stuart St. at Charles St.,Member
1,0 days 00:05:13.200000,2011-11-30 23:56:00,2011-12-01 00:01:00,C32008,Boylston at Fairfield,D32011,Stuart St. at Charles St.,Casual
2,0 days 00:18:31.430000,2011-11-30 23:18:00,2011-11-30 23:36:00,A32009,Tremont St / W Newton St,D32006,Lewis Wharf - Atlantic Ave.,Member
3,0 days 00:21:53.487000,2011-11-30 23:15:00,2011-11-30 23:37:00,A32001,Union Square - Brighton Ave. at Cambridge St.,D32005,Boston Public Library - 700 Boylston St.,Member
4,0 days 00:05:45.115000,2011-11-30 22:59:00,2011-11-30 23:05:00,B32008,Mayor Martin J. Walsh - 28 State St.,D32006,Lewis Wharf - Atlantic Ave.,Member


In [296]:
pre_2015_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2762938 entries, 0 to 2762937
Data columns (total 8 columns):
 #   Column              Dtype          
---  ------              -----          
 0   duration            timedelta64[ns]
 1   started_at          datetime64[ns] 
 2   ended_at            datetime64[ns] 
 3   start_station_id    object         
 4   start_station_name  object         
 5   end_station_id      object         
 6   end_station_name    object         
 7   member_casual       object         
dtypes: datetime64[ns](2), object(5), timedelta64[ns](1)
memory usage: 168.6+ MB


In [ ]:
# 3
# TODO

In [ ]:
# now let's look again at the null value rows' durations to see if they are removable outliers/exceptions

big_df.loc[null_indices]

In [ ]:
# if we remove null values then what are the stats

big_df.loc[null_indices]

In [ ]:
big_df.head()

In [ ]:
# takes like 22 sec to run

# seem sus indeed! so we gonna drop those null rows

clean_df = big_df[big_df.notna().all(axis=1)]

In [ ]:
clean_df.shape[0]

yas we got 27,035,163 clean rows now

In [ ]:
clean_df.info()

In [ ]:
clean_df.describe()

uhhhh min of like negative duration seems sussy... BRUH max duration is 492 days

GOOTA INVESTIGATE

In [ ]:
from datetime import timedelta

# investgiate durationnnnnn


# to DOOOOOO
greater_than_day_duration_indices = clean_df[clean_df['duration'] > timedelta(days=1)].index.tolist()

clean_df[clean_df['duration'] > timedelta(days=1)]


# gonna filter out lol

In [ ]:
# also check durations < 0
negative_duration_indices = clean_df[clean_df['duration'] < timedelta(days=0)].index.tolist()
clean_df[clean_df['duration'] < timedelta(days=0)] 

# lol deleting these two bc no es possible

In [ ]:
original_df.dtypes

In [ ]:
original_df.loc[greater_than_day_duration_indices]

# just checking no issues with like interpreting dates and that these are indeed > 24 hour durations - confirmed

In [ ]:
original_df.loc[negative_duration_indices]
# same but checking for negative duration --> confirmed!

In [ ]:
# now confirmed outliers, removing duration outliers (<0 or >24 hours)

clean_df = clean_df[(clean_df['duration'] < timedelta(days=1)) & (clean_df['duration'] > timedelta(0))]

In [ ]:
clean_df.info()
# yay final df!

# however dtypes are wrong so try to fix that

In [ ]:
clean_df.describe()

looks all good! only thing to consider doing is deleting the max outliers (within 24 hous but like > x std devs above median or something)

In [ ]:
clean_df.isna().sum()

YAY no null values. and duration seems reasonable. let's double check unique values of each column now as final preprocessing

In [ ]:
# check unique values for categorical cols like station names and ids and member_casual

clean_df.columns

### plot distribution of member_casual tyepes and duration histogram
so first need to turn **Customer -> casual** and **Subscriber -> member**

In [ ]:
# need to do that first

In [ ]:
from matplotlib import pyplot as plt
fig, axis = plt.subplots()
import seaborn as sns
sns.countplot(x='member_casual', data=clean_df, ax=axis) # TODO make this a pie chart

# this takes forever, is ther a faster way?

we see that majority of rides are by members which makes sense. let's see the trend over time

In [ ]:
# TODO: see member vs. casual distribution each year

### then plot most popular departure and arrival stations


In [ ]:
hi

In [ ]:
# need to figure out the station name situation lol
# 
start_names = clean_df["start_station_name"].unique().sort()
end_names = clean_df["end_station_name"].unique().sort()
start_id = clean_df["start_station_id"].unique().sort() # errorrrrrrrrrrrrrr 
end_id = clean_df["end_station_id"].unique().sort()

### plot distributions of stations (start, end) popularity......
pending above code cell

# cast all coordinates to floats? can floats be negative (yes)

then map all the stations onto a map of boston... 

In [ ]:
# converting coordinates to floats


In [ ]:
start_names

In [ ]:
from collections import Counter
# start_names == end_names # lol compare them to see if they are identical or nah
# start_id == end_id

print(Counter(start_names) == Counter(end_names))
print(Counter(start_id) == Counter(end_id))

In [ ]:
# rename columns
# standardize formatting e.g. starttime

In [ ]:
# eda techniques

to do ideas for later

- turn things into functions, like datetime transforming and getting duration etc.

# EDA